Importar librerias

In [1]:
import os
import sys
import random
from src.config import *
from src.data import *


Mendeley 224

In [2]:
img = explorar_split_data(MENDELEY_OAI_224_SPLIT_PATH)


╒═════════╤══════╤══════╤══════╤═════╤═════╕
│ Clase   │    0 │    1 │    2 │   3 │   4 │
╞═════════╪══════╪══════╪══════╪═════╪═════╡
│ train   │ 2286 │ 1046 │ 1516 │ 757 │ 173 │
├─────────┼──────┼──────┼──────┼─────┼─────┤
│ val     │  328 │  153 │  212 │ 106 │  27 │
├─────────┼──────┼──────┼──────┼─────┼─────┤
│ test    │  639 │  296 │  447 │ 223 │  51 │
╘═════════╧══════╧══════╧══════╧═════╧═════╛


In [3]:
img = explorar_data(MENDELEY_EXPERT1_PATH)

╒═════════╤════════════╕
│   Clase │   Cantidad │
╞═════════╪════════════╡
│       0 │        514 │
├─────────┼────────────┤
│       1 │        477 │
├─────────┼────────────┤
│       2 │        232 │
├─────────┼────────────┤
│       3 │        221 │
├─────────┼────────────┤
│       4 │        206 │
╘═════════╧════════════╛


In [4]:
img = explorar_data(MENDELEY_EXPERT2_PATH)

╒═════════╤════════════╕
│   Clase │   Cantidad │
╞═════════╪════════════╡
│       0 │        503 │
├─────────┼────────────┤
│       1 │        488 │
├─────────┼────────────┤
│       2 │        232 │
├─────────┼────────────┤
│       3 │        221 │
├─────────┼────────────┤
│       4 │        206 │
╘═════════╧════════════╛
